<a href="https://colab.research.google.com/github/robindoering86/Project2-Kickstarter_success/blob/master/Kickstarter_Project_Success_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

3. Kickstarter Project Success - Robin & Peter, Manuel & Simon


In recent years, the range of funding options for projects created by individuals and small companies has expanded considerably. In addition to savings, bank loans, friends & family funding and other traditional options, crowdfunding has become a popular and readily available alternative. 

Kickstarter, founded in 2009, is one particularly well-known and popular crowdfunding platform. It has an all-or-nothing funding model, whereby a project is only funded if it meets its goal amount; otherwise no money is given by backers to a project.
A huge variety of factors contribute to the success or failure of a project — in general, and also on Kickstarter. Some of these are able to be quantified or categorized, which allows for the construction of a model to attempt to predict whether a project will succeed or not. The aim of this project is to construct such a model and also to analyse Kickstarter project data more generally, in order to help potential project creators assess whether or not Kickstarter is a good funding option for them, and what their chances of success are.


# Project 2: Kickstarter Project Success

## Import libraries

In [0]:
# Pandas as NumPy
import pandas as pd
import numpy as np

# Scikit-Learn
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, roc_curve, confusion_matrix 

# Pyplot
import matplotlib.pyplot as plt
import seaborn as sns


from sklearn.model_selection import train_test_split


# Others
from os import listdir
import time


## Read in the data

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
# For local execution
#base_path = './data/'

# For google colab execution
#base_path = '/content/drive/My Drive/Colab Notebooks/data/' # peter
base_path = '/content/drive/My Drive/Kickstarter_data/data/' # robin
data = pd.concat([pd.read_csv(base_path+x) for x in sorted(listdir(base_path))], axis=0)
data_cp = data.copy()

In [0]:
# Create df for the second half of colu,ns
data_sh = data.iloc[:, 17:]

In [0]:
data.shape

In [0]:
data.isna().sum()

In [0]:
data.columns

In [0]:
data.describe()

Description of columns would be helpful!

In [0]:
data.info()

In [0]:
# significant amout of missing data in these columns, ignore for now
# Define a list columns to drop
droplist = []
droplist = droplist+['friends', 'is_backing', 'is_starred', 'permissions']

### The predicted variable: 'state'

In [0]:
data_sh['state'].describe()

In [0]:
data_sh['state'].value_counts()

In [0]:
# Calculate percentage of 'state' value that are not either 'successful' or 'failed'
abs(len(data_sh[(data_sh['state'] == 'successful') | (data_sh['state'] == 'failed')]) - len(data_sh)) / len(data_sh)

'state' has 5 different values: 'successful', 'failed', 'canceled', 'live', and 'suspended'. If we want binary target class to be binary, we would loose approx 8% of the dataset.

In [0]:
data_sh[(data_sh['state'] == 'successful') | (data_sh['state'] == 'failed')].info()

In [0]:
# New dataframe which contains only entries which have the state 'successful' or 'failed'
filtered_df = data[(data['state'] == 'successful') | (data['state'] == 'failed')].copy()
filtered_df.reset_index(inplace=True)

In [0]:
filtered_df.columns

### The 'is_starrable' column
is_starrable - whether or not a project can be starred (liked and saved) by users

In [0]:
col = 'is_starrable'

In [0]:
filtered_df[col].head()

In [0]:
filtered_df[col] = filtered_df[col].astype(bool)

In [0]:
filtered_df[col].describe()

In [0]:
filtered_df[col].value_counts()

In [0]:
# Any missing values?
filtered_df[col].isnull().sum()

### The 'launched_at' column

launched_at - date and time of when the project was launched for funding

In [0]:
col = 'launched_at'
filtered_df[col].head()

In [0]:
# Make human-readable dates out of timestamp
filtered_df[col] = pd.to_datetime(filtered_df[col], unit='s')

In [0]:
#data_sh['launched_at'].iloc[0].dayofweek

filtered_df['launch_weekday'] = [x.day_name() for x in filtered_df['launched_at']]
filtered_df['launch_weekday'] = filtered_df['launch_weekday'].astype(str)
filtered_df['launch_weekday'].dtype

In [0]:
filtered_df['launch_year'] = [x.year for x in filtered_df['launched_at']]
filtered_df['launch_year'] = filtered_df['launch_year'].astype(int)
filtered_df['launch_year'].dtype

In [0]:
filtered_df['launch_month'] = [x.month for x in filtered_df['launched_at']]
filtered_df['launch_month'] = filtered_df['launch_month'].astype(int)
filtered_df['launch_month'].dtype

In [0]:
filtered_df['launch_day'] = [x.day for x in filtered_df['launched_at']]
filtered_df['launch_day'] = filtered_df['launch_day'].astype(int)
filtered_df['launch_day'].dtype

In [0]:
# Create a new column for launch hour
filtered_df['launch_hour'] = [x.hour for x in filtered_df['launched_at']]
filtered_df['launch_hour'] = filtered_df['launch_hour'].astype(int)
filtered_df['launch_hour'].dtype

In [0]:
# TODO: Create bins for launch time

In [0]:
# Compare launch hour of successfull and failed projects
plt.hist(filtered_df[filtered_df['state'] == 'successful']['hour'], bins=12, alpha=0.6)
plt.hist(filtered_df[filtered_df['state'] == 'failed']['hour'], bins=12, alpha=0.6, color='r')

In [0]:
filtered_df['year'].unique()

In [0]:
# TODO: check whether launch time has changed over years, see whether it changes over the months

### The 'location' column

In [0]:
col = 'location'

filtered_df[col].dtype

In [0]:
filtered_df[col].head(10)

Looks like str representantion of a dictionary, turn into dict first.

In [0]:
#start =time.time()
del_idx = []
count = 0

for idx, x in enumerate(filtered_df[col]):

    try:
        filtered_df.at[idx, col] = eval(x.replace('false', 'False').replace('true', 'True').replace('null', 'None'))
        #eval(x.replace('false', 'False').replace('true', 'True').replace('null', 'None'))

    except:
        filtered_df.drop(idx, axis=0, inplace=True)
        del_idx.append(idx)
#end = time.time()
#print(end-start)
filtered_df.reset_index(inplace=True)

In [0]:
filtered_df[col][0]

In [0]:
# Count number of unique locations

#len(set([x['name'] for x in filtered_df[col]]))

filtered_df['city'] = [x['name'] for x in filtered_df[col]]

In [0]:
# Top 20 cities by project frequency

filtered_df['city'].value_counts().head(20).plot(kind='bar')

In [0]:
# Top 20 non-US cities

filtered_df['city'][filtered_df['location']['country'] != 'US'].value_counts().head(20).plot(kind='bar')

In [0]:
filtered_df['location'][0]['country']

In [0]:
plt.figure(figsize=(15,7))
data['country'].value_counts().plot(kind='bar')

### The 'name' column

In [0]:
col = 'name'
filtered_df[col].describe()

In [0]:
filtered_df[[col]].isna().sum()

No missing values but apparently 24000 non-unique entries / names. This certainly requires further addressing.

In [0]:
# Look at the distribution of project name lengths
plt.hist([len(x) for x in filtered_df[col]], bins=30)

In [0]:
# Look into:
# lenght of name vs successful / failed funding
# sentiment analysis vs. funding success

### The 'photo' column

In [0]:
col = 'photo'
filtered_df[col].describe()

In [0]:
filtered_df[col].head()

Another column that looks like str reprensentation of a dict

In [0]:
filtered_df[col].isna().sum()

In [0]:
eval(filtered_df[col][0])['1536x864']

In [0]:
# Turn strings into dicts and check for exceptions errors

del_idx = []
count = 0
for idx, x in enumerate(filtered_df[col]):

    try:
        filtered_df.at[idx, col] = eval(x.replace('false', 'False').replace('true', 'True').replace('null', 'None'))
        #eval(x.replace('null', 'None'))
    except:
        count += 1
        del_idx.append(idx)
print(count)

TODO: Analyse 'photos'

### The 'pledged' column

pledged data is given in native currencies. **So can be dropped in favour of 'usd_pledged' which gives pledged amount in same currency (USD)**

In [0]:
col = 'pledged'
filtered_df[col].dtypes

In [0]:
filtered_df[col].describe()

In [0]:
filtered_df[col].isna().sum()

No missing values 

In [0]:
filtered_df[col].value_counts()


15003 projects that pledged 0 USD?

In [0]:
droplist.append(col)
#filtered_df.drop('pledged', inplace=True, axis=1)

### The 'profile' column

In [0]:
col = 'profile'

filtered_df[col].head()

Yet another column with str reprensentation of a dict. Lets look into one entry:

In [0]:
entry = eval(filtered_df[col][0].replace('null', 'None').replace('false', 'False').replace('true', 'True'))
entry

In [0]:
list(entry.keys())

In [0]:
# Check which of the entries of the 'profile' column already exist as separate column s

for e in list(data.columns):
    if e in list(entry.keys()):
        print(e)

In [0]:
## TODO: Check whether contents also match

Seems like a repetition of other columns. And some additional information. Well drop the column for now.

In [0]:
droplist.append(col)

### The 'slug' column

In [0]:
col = 'slug'
filtered_df[col].head()

In [0]:
filtered_df['name'].head()

At first glance, this looks like the 'name' column in all lower case and with hyphens instead of whitespaces.

In [0]:
filtered_df[col].describe()

In [0]:
[filtered_df['name'].str.lower().replace(' ', '-').replace(':', '') == filtered_df[col]]

In [0]:
li = []
for i in range(10):
    
    li.append(filtered_df['name'][i].lower().replace(' ', '-').replace(':', '') == filtered_df['slug'][i])
    

In [0]:
filtered_df[col].str.len().max()

In [0]:
filtered_df[col][2]

In [0]:
filtered_df['name'][2].lower().replace(' ', '-').replace(':', '')

In [0]:
filtered_df[col].str.lower().replace(' ', '-').replace(':', '')

droplist.append(col)

### The 'source_url' column

In [0]:
col = 'source_url'
filtered_df[col][1000]

This is just an url to the category of the project, drop.

In [0]:
droplist.append(col)

### The 'staff_pick' column

staff_pick - whether a project was highlighted as a staff_pick when it was launched/live

Seems like an interesting variable. 

In [0]:
col = 'staff_pick'

filtered_df[col].head()

In [0]:
filtered_df[col].value_counts()

In [0]:
round(25940/(25940+166509)*100, 2)

13.48 % of projects where highlighted as staff_pick, when they were launched/live.

### The 'state_changed_at' column
state_changed_at - date and time of when a project's status was changed (same as the deadline for successful and failed projects)

In [0]:
col = 'state_changed_at'

filtered_df[col].head()

In [0]:
# Conver to datetime object
filtered_df[col] = pd.to_datetime(filtered_df[col], unit='s')

### The 'static_usd_rate' column

static_usd_rate - conversion rate between the original currency and USD

In [0]:
col = 'static_usd_rate'

filtered_df[col].head()

Assume column is just a mulitplier to convert 'pledge' to 'usd_plegded', drop.

In [0]:
droplist.append(col)

### The 'urls' columns
urls - url to the project's page

In [0]:
col = 'urls'

filtered_df[col].head()

In [0]:
eval(filtered_df[col][0])

Just links to the project page, drop.

In [0]:
droplist.append(col)

### The 'usd_pledged' column
usd_pledged - amount pledged in USD

In [0]:
col = 'usd_pledged'

filtered_df[col].head()

In [0]:
filtered_df[col].describe()

In [0]:
filtered_df[col].isna().sum()

In [0]:
filtered_df[col].value_counts()

In [0]:
filtered_df[filtered_df[col] == np.float(0)]['state'].value_counts()

### The 'usd_type' column
usd_type - domestic or international

In [0]:
col = 'usd_type'

filtered_df[col].head()

In [0]:
filtered_df[col].unique()

In [0]:
filtered_df[col].isna().sum()

38 missing values, but can be ignored as columns will be dropped anyways.

In [0]:
droplist.append(col)

In [0]:
droplist

In [0]:
filtered_df.shape

In [0]:
filtered_df.drop(droplist, axis=1, inplace=True)

In [0]:
filtered_df.shape

In [0]:
filtered_df.columns

###The rest of the columns

In [0]:
# split creator column entry in sub_columns
df_tot = filtered_df
import json 
#df_creator = pd.DataFrame(json.loads(df_tot.creator.values))
creator_dlist = list()
idx_drop = list()
backup = None
#print(json.loads(df_tot.creator.values[0]))
count = 0
for idx, x in enumerate(df_tot.creator.values):
  try:
    creator_dlist.append(json.loads(x))
    #df_tot.creator.values[idx] = json.loads(x)
  except:
    # catches the other cases with the exception of 7 cases
    a = x.index("\"name\":")
    b = x.index("\"", a + 8)
    x = x[:b] + "'" + x[b + 1:]
    c = x.index("\"", b)
    x = x[:c] + "'" + x[c + 1:]
    try:
      creator_dlist.append(json.loads(x))
    except:
      print(a,b,c)
      count += 1
      idx_drop.append(idx)
print(count)
df_creator = pd.DataFrame(creator_dlist)
df_creator.columns = ['crea_' + x for x in df_creator.columns]
df_creator.info()
df_tot2 = df_tot.drop(idx_drop, axis = 0).reset_index()


In [0]:
# drop two columns with only null entry
df_creator.drop(['crea_is_registered', 'crea_chosen_currency'], axis = 1, inplace = True)
df_creator.head()

In [0]:
category_dlist = list()
count = 0
for idx, x in enumerate(df_tot2.category.values):
  try:
    category_dlist.append(json.loads(x))
  except:
    count += 1

print(count)

df_category = pd.DataFrame(category_dlist)
df_category.columns = ['cat_' + x for x in df_category.columns]
df_category.info()

In [0]:
df_category.cat_name = df_category.cat_name.str.lower()
df_category.cat_name = df_category.cat_name.str.replace(" ", "_")
df_category.columns = ['cat_super' if x=='cat_slug' else x for x in df_category.columns]
df_category.cat_super = [str(x).split(sep = '/')[0] for x in df_category.cat_super.values]
df_category.cat_super = df_category.cat_super.replace(" ", "_")
df_category.tail()

In [0]:
df_tot3 = pd.concat([df_tot2, df_creator, df_category],axis = 1)
df_tot3.drop(['creator', 'category'], inplace = True, axis = 1)
df_tot3.info()

In [0]:
df_tot3.created_at = pd.to_datetime(df_tot3.created_at, unit='s')
df_tot3.deadline = pd.to_datetime(df_tot3.deadline, unit='s')

In [0]:
df_tot3.id = df_tot3.id.astype('category')

In [0]:
df_tot3.blurb = df_tot3.blurb.fillna('')
df_tot3.blurb = df_tot3.blurb.astype(str)
blurb_wc = [len(str(x).lower().split()) for x in df_tot3.blurb]
blurb_un_words = [set(str(x).lower().split()) for x in df_tot3.blurb]
blurb_un_words = set.union(*blurb_un_words)
print(len(blurb_un_words))
print(set(blurb_wc))
df_tot3['blurb_wordcount'] = blurb_wc
#df_tot3[df_tot3.blurb.isna()]
# keyword analysis in blurb as bonus

In [0]:
df_tot3.country = df_tot3.country.astype('category')
df_tot3.currency = df_tot3.currency.astype('category')
df_tot3.currency_symbol = df_tot3.currency_symbol.astype('category')
df_tot3.current_currency = df_tot3.current_currency.astype('category')

In [0]:
# deletes columns that contain only little information
df_tot4 = df_tot3.drop(['friends', 'is_backing', 'is_starred', 'permissions'], axis = 1)

In [0]:
df_tot4.crea_slug = df_tot4.crea_slug.fillna('')
df_tot4.cat_parent_id = df_tot4.cat_parent_id.fillna(0)

In [0]:
# list of project list according to unique ids
id_groupby = df_tot4.groupby('id')

id_groupby2 = id_groupby.size().reset_index(name='counts')
indexid = id_groupby2[id_groupby2.counts > 1].index
iddouble_list = id_groupby2[id_groupby2.counts > 1].id.unique()


In [0]:
double_entries_sublist = [df_tot4[df_tot4.id == x].index[1:] for x in iddouble_list]
double_entries_sublist_flat = list()
for y in double_entries_sublist:
  for x in y:
    double_entries_sublist_flat.append(x)
#double_entries_sublist_flat

In [0]:
df_tot5 = df_tot4.drop(double_entries_sublist_flat, axis = 0)

In [0]:
# list of project list of unique creator
unique_crea_set = set(df_tot5.crea_id.unique())
#crea_id_entry

creaid_groupby = df_tot5.groupby('crea_id')


creaid_groupby2 = creaid_groupby.size().reset_index(name='counts')
indexcreaid = creaid_groupby2[creaid_groupby2.counts > 1].index
creaiddouble_list = creaid_groupby2[creaid_groupby2.counts > 1].crea_id.unique()
# number of creators with x projects
multi_proj = list()
x_list = list()
for x in range(1,75):
  a = len(creaid_groupby2[creaid_groupby2.counts == x].crea_id.unique())
  multi_proj.append(a)
  x_list.append(x)
  #print(x, np.log10(a))

In [0]:
df_tot5.cat_super.hist(bins = len(df_tot5.cat_super.unique()), figsize = (17,17), grid = False)

In [0]:
df_tot5[df_tot5.state == 'successful'].cat_super.hist(bins = len(df_tot5.cat_super.unique()), figsize = (17,17), grid = False)

In [0]:
df_tot5[df_tot5.state == 'failed'].cat_super.hist(bins = len(df_tot5.cat_super.unique()), figsize = (17,17), grid = False)

In [0]:
init_month = [x.astype('datetime64[M]').astype(int) % 12 + 1 for x in df_tot5.created_at.values]
end_month = [x.astype('datetime64[M]').astype(int) % 12 + 1 for x in df_tot5.deadline.values]
init_year = [x.astype('datetime64[Y]').astype(int) + 1970 for x in df_tot5.created_at.values]
end_year = [x.astype('datetime64[Y]').astype(int) + 1970 for x in df_tot5.deadline.values]
init_month_succ = [x.astype('datetime64[M]').astype(int) % 12 + 1 for x in df_tot5[df_tot5.state == 'successful'].created_at.values]
end_month_succ = [x.astype('datetime64[M]').astype(int) % 12 + 1 for x in df_tot5[df_tot5.state == 'successful'].deadline.values]
init_year_succ = [x.astype('datetime64[Y]').astype(int) + 1970 for x in df_tot5[df_tot5.state == 'successful'].created_at.values]
end_year_succ = [x.astype('datetime64[Y]').astype(int) + 1970 for x in df_tot5[df_tot5.state == 'successful'].deadline.values]
print(df_tot5.created_at.values[1], df_tot5.created_at.values[1].astype('datetime64[M]').astype(int) % 12 + 1)
init_year[1]

In [0]:
plt.hist(init_month)

In [0]:
plt.hist(init_month_succ)

In [0]:
plt.hist(end_month)

In [0]:
plt.hist(end_month_succ)

In [0]:
plt.hist(init_year)

In [0]:
plt.hist(init_year_succ)

In [0]:
plt.hist(end_year)

In [0]:
plt.hist(end_year_succ)

In [0]:
stategroupby = df_tot5.groupby('state').count().blurb
statedict = dict(stategroupby)
plt.bar(statedict.keys(), statedict.values());

In [0]:
!pip install gender-guesser
import gender_guesser.detector as gender

In [0]:
d = gender.Detector()
gender_list = list()
for x in df_tot5.crea_name.values:
  gender_list.append(d.get_gender(x.split()[0]))
gender_list
male_count = 0
mmale_count = 0
female_count = 0
mfemale_count = 0
unknown_count = 0
unclear_count = 0

for x in gender_list:
  if x == 'male':
    male_count += 1
  elif x == 'mostly_male':
    mmale_count += 1
  elif x == 'female':
    female_count += 1
  elif x == 'mostly_female':
    mfemale_count += 1
  elif x == 'unknown':
    unknown_count += 1
  else:
    unclear_count += 1
print(male_count, mmale_count, female_count, mfemale_count, unknown_count, unclear_count)
for idx, x in enumerate(gender_list):
  if x == 'mostly_male':
    gender_list[idx] = 'male'
  elif x == 'mostly_female':
    gender_list[idx] = 'female'
  elif x == 'andy':
    gender_list[idx] = 'unknown'

In [0]:
df_tot5.describe()

In [0]:
df_tot5['gender'] = gender_list

In [0]:
suclist = list()
for x in df_tot5.state.values:
    if x == 'successful':
        suclist.append(1)
    else:
        suclist.append(0)
df_tot5['success'] = suclist

In [0]:
corrm = df_tot5.corr()

In [0]:
plt.figure(figsize = (15,15))
sns.heatmap(corrm)

In [0]:
df_tot5.gender = df_tot5.gender.astype('category')
df_tot5.cat_super = df_tot5.cat_super.astype('category')
df_tot5.cat_parent_id = df_tot5.cat_parent_id.astype('category')
df_tot5.city = df_tot5.city.astype('category')
df_tot5.gender = df_tot5.gender.astype('category')

In [0]:
df_tot6 = df_tot5.drop(['level_0', 'index', 'blurb', 'converted_pledged_amount',\
                        'currency', 'currency_symbol', 'currency_trailing_code',\
                        'id', 'name', 'slug', 'photo', 'state', 'state_changed_at',\
                        'crea_name', 'crea_avatar', 'crea_urls', 'crea_slug',\
                        'cat_color', 'cat_urls', 'cat_parent_id', 'cat_id'], axis = 1)